In [2]:
import tensorflow as tf
import numpy as np

# if this import fails, add the folder Space-RNN-GRU to sys.path
# import sys
# sys.path.append("/path/to/folder/Space-RNN-GRU/")

from mdrnn import MultiDimensionalRNN

tf.__version__

'2.0.0'

## Dummy example to test MDRNN flow

#### <center>Recurrent function to compute the hidden state at position i,j</center>

$$\vec{h}_{ij}=f(\vec{h}_{i-1,j},\vec{h}_{i,j-1}, \vec{h}_{i-1,j-1}, \vec{s}_{ij})$$

In [3]:
# normal python plus numpy implementation of this recursion with a simple computation (sum all the previous states plus the entry)

def cell(left_state, top_state, diagonal_state, entry):
    return left_state + top_state + diagonal_state + entry


def recursive_mdrnn(input_matrix, hidden, i,j):
    
    if i<0 or j<0:
        return 0
    
    if hidden[i,j] !=- 1: # MEMOIZATION to speed up the recursion (only computes once)
        return hidden[i,j]
    
    # set
    hidden[i,j] = cell(recursive_mdrnn(input_matrix, hidden, i, j-1),
                       recursive_mdrnn(input_matrix, hidden, i-1, j),
                       recursive_mdrnn(input_matrix, hidden, i-1, j-1),
                       input_matrix[i,j])
    
    return hidden[i,j]

In [4]:
# keras tensorflow model with same computation
SHAPE = (5,5,1)

class SumCell(tf.keras.layers.Layer):
    def call(self, x, states):
        left_state = states[0]
        top_state = states[1]
        diagonal_state = states[2]
        
        return left_state + top_state + diagonal_state + x

def create_model(shape, show_summary = True):
    model = tf.keras.Sequential()
    model.add(MultiDimensionalRNN(SumCell(), inital_state=0.0, input_shape=shape))
    
    if show_summary:
        model.summary()
    return model

model = create_model(SHAPE)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
multi_dimensional_rnn (Multi (None, 1)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [5]:
# define matrix
matrix = np.zeros(SHAPE)
matrix[1,2] = 1
matrix[2,3] = 1

print(matrix[:,:,0])

[[0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


## Compare results with two solutions

In [6]:

hidden_states_matrix = np.zeros(SHAPE) - 1
r0 = recursive_mdrnn(matrix, hidden_states_matrix, SHAPE[0]-1, SHAPE[1]-1)
print("Recursive solution:", r0)
print(hidden_states_matrix[:,:,0])

r1 = model.predict(np.array([matrix]))
print("Tensorflow solution:", r1)


Recursive solution: [30.]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  0.  1.  4.  6.]
 [ 0.  0.  1.  6. 16.]
 [ 0.  0.  1.  8. 30.]]
Tensorflow solution: [[30.]]


In [7]:

# some random matrix texts
N = 100
for i in range(N):
    
    m = np.random.randint(0,10, SHAPE)

    hidden_states_matrix = np.zeros(SHAPE) - 1
    r0 = recursive_mdrnn(m, hidden_states_matrix, SHAPE[0]-1, SHAPE[1]-1)

    r1 = model.predict(np.array([m]))
   
    assert r0[0]==r1[0][0]

print("No errors")

No errors


In [8]:
from tensorflow.keras import backend as K

# some random matrix texts with random shapes

N = 100
for i in range(N):
    
    shape_rows = np.random.randint(2,10, ())
    shape_cols = np.random.randint(2,10, ())

    shape = (shape_rows, shape_cols, 1)

    m = 0.02*np.random.normal(size=shape) -0.01

    hidden_states_matrix = np.zeros(shape) - 1
    r0 = recursive_mdrnn(m, hidden_states_matrix, shape[0]-1, shape[1]-1)
    
    # need to reacreate the model due to shape incompatibility
    K.clear_session()
    model = create_model(shape, show_summary=False)
    r1 = model.predict(np.array([m]))
   
    assert np.round(r0[0])==np.round(r1[0][0])

print("No errors")

No errors
